# One to One Matching

In [4]:
import os
import cv2
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import math
import operator
from tqdm import tqdm
import torch.nn.functional as F
from scipy.optimize import linear_sum_assignment

# Load feature & keypoints

In [5]:
train_features = torch.load("train_features.pth")
print(train_features.shape)
train_keypoints = torch.load("train_keypoints.pth")
print(train_keypoints.shape)

test_features = torch.load("test_features.pth")
print(test_features.shape)
test_keypoints = torch.load("test_keypoints.pth")
print(test_keypoints.shape)

torch.Size([100, 17, 128])
torch.Size([100, 17, 2])
torch.Size([50, 17, 128])
torch.Size([50, 17, 2])


# Mathching

In [23]:
kps_num = 5
ground_truth = [int(i/25) for i in range(50)]
result = []
dataset_size = 50
dataset = []
for i in range(dataset_size):
    dataset.append(i)
    dataset.append(i+50)

#print(dataset)

for q in range(50):
    total_simi = []
    img1 = test_features[q]
    img1 = img1.view(kps_num, 1, 128).expand(kps_num, kps_num, 128).contiguous().view(kps_num*kps_num, 128)
    for i in dataset:
        img2 = train_features[i]


        img2 = img2.view(1, kps_num, 128).expand(kps_num, kps_num, 128).contiguous().view(kps_num*kps_num, 128)

        simi = (1 + F.cosine_similarity(img1, img2))/2
        simi = simi.view(kps_num, kps_num)
        cost = 1 - simi

        row_ind, col_ind = linear_sum_assignment(cost.cpu().numpy())
        total_simi.append(simi[row_ind, col_ind].sum())

    coast = 0
    forest = 0

    for i in range(dataset_size*2):
        if i%2 == 0: 
            coast += total_simi[i]
        else:
            forest += total_simi[i]
    
    if(coast > forest):
        result.append(0)
    else:
        result.append(1)
        

[0, 50, 1, 51, 2, 52, 3, 53, 4, 54, 5, 55, 6, 56, 7, 57, 8, 58, 9, 59]


# Evaluation

In [24]:
k = 0
for i in range(len(ground_truth)):
    if (result[i] == ground_truth[i]):
        k += 1
precise = k/len(ground_truth)
print(precise)

0.84
